# Set kernel to: `py-pythonEnv`
If this kernel is not available, see the Kernel Installation Instructions section located in at the bottom of this Notebook.

# Data Science on the C3 AI Suite

## C3DS - Monitoring \& Debugging Jobs \& Queues

#### For questions, comments, or concerns, please email C3.ai Academy Team \< training@c3.ai \>, discuss with your Trainer, or leave your thoughts in the Google Forms at the end of this module.

### Table of Contents 

__[C3DS Managing ML Models with Jobs](#part1)__

* [Section 1 - Launch MapReduce Job "On the Fly"](#1)
    * [A. Create MLSerialPipeline](#1A)
    * [B. mapFunction](#1B)
    * [C. reduceFunction](#1C)
    * [D. MapReduce Job Specifications](#1D)    
    
__[C3DS Monitor, Debug Jobs \& Queues](#part2)__

* [Section 2 - Launching the MapReduce Job \& Debugging](#2)   


* [Section 3 -  Run the Full MapReduce Job](#3)   


* [Section 4 - Try Other Jobs (Optional)](#4)   


[Appendix - Kernel Installation Instructions](#apdx)

# C3DS Manage ML Models with Jobs <a class="anchor" id="part1">

## Exercise Goal

In this exercise, you will experience map reduce jobs \& queue monitoring. One of the benefits of using ML Pipelines is that data preparation and model training are performed using server/cluster side resources (in the background and highly parallelized - instead of your notebook resources). In the process, you'll also learn how to monitor queues and perform some basic debugging. 
    
If you have not already - watch the video of stepping through the notebook along with code explanations. Then, try executing the map reduce job and debugging yourself.

## Section 1 - Configure MapReduce Job <a class="anchor" id="1"></a>

In this assignment, we will launch a MapReduce job to train train three different models, one for each `SmartBulb` manufacturer. 

The following code snippets (pictured in sections A..D) are located in `src/jobs/JSMapReduceTraining_BUGGY.js`. Copy this code into **Chrome Developer Snippets** and we will be incrementally running and debugging this code. Study the code to ensure you understand what the code is (trying) to do.
    
The script is composed of 4 parts:
1. Create MLSerialPipeline
2. Define mapFunction
3. Define reduceFunction
4. Define MapReduce job specification (which combines all the above)
    
> Note: This is an exercise on debugging, a valuable skill that is best learned through experience, and one that requires patience and perseverence. **Hang in there, and good luck!**

### A. Create MLSerialPipeline <a class="anchor" id="1A">

This part of the script creates the `MLSerialPipeline`, which is made of two steps:
1. A normalization step with `standardScaler`;
2. A classification step with `logiscticRegression`.

> Note: Code available from: `src/jobs/JSMapReduceTraining_BUGGY.js`


![image](http://learnc3.ai/wp-content/uploads/2020/01/Setup.png)

The metric used to score the model is the **accuracy**. In the last step, after defining the pipeline, we `upsert` it into the platfrom (becomes an instance of `MLSerialPipeline` type). 

After executing the above code in Console, run, `c3Grid(MLSerialPipeline.fetch({filter: "id == 'logisticRegressionMR'"}))`.  You will see the model as shown below:


![image](http://learnc3.ai/wp-content/uploads/2020/01/untrainedModel.png)

### B. mapFunction <a class="anchor" id="1B">

    
This map function will perform an `evalMetric` call on the metrics needed by the model (including the metric for target label) and return a dictionary where the `key` is the SmartBulb **manufacturer** and the `value` is the dataset made from the `evalMetric` call.

> Note: Code available from: `src/jobs/JSMapReduceTraining_BUGGY.js`
    
![image](http://learnc3.ai/wp-content/uploads/2020/01/MapFunction.png)

When we invoke *mapFunction()* from the JSMapReduce job (in section 1D) - the map function will be provided 3 parameters:

1. **batch** - information about the work to be performed
2. **objs** - a list of "things" to perform the computation on. 
3. **job** - information about the overall job, such as context (execution variables)

**Explanation:**

Recall that if we have many objects to process (say, 1 million) - the map reduce framework (provided by the C3.ai platform) will breakup the dataset into many chunks of data. Each "mapper" (executed on a worker node) gets one batch of data, applies the map function, and returns the (partial) answer derived from its assigned batch of data. The batch of data is provided to the map function as an array of objects in the `objs` parameter. 

If the total number of objects to be processed is X and the batch size for the map reduce job is set to N - we can expect that the objs will be of size X - and the map reduce queue will have X/N entries when the job is initialized.

The `job` parameter provides additional information for each mapper's computation. The context object (passed in as `job.context`) is user defined and is a way for us to pass variables into the map and reduce functions. 

In the above example - the mapper is going to perform SmartBulb.evalMetric() which itself requires some parameters. We don't want these variables to be hard coded. Instead, we desire the ability to run this at any time using different **features** and **target labels**, and differing values for `start`, `end`,`interval`, and `ids` (of SmartBulbs). All this is passed in by `job.context` - which is defined at runtime when we actually invoke the map reduce job.




### C. reduceFunction <a class="anchor" id="1C">

    
This **reduce** function concatenates the datasets that have the **same manufacturer**, trains the model on this dataset, then *upserts* the trained model to the platform (making it available to be invoked by other applications via REST APIs). Run this section of code in console.
    
> Note: Code available from: `src/jobs/JSMapReduceTraining_BUGGY.js`. 

    
![image](http://learnc3.ai/wp-content/uploads/2020/01/ReduceFunction.png)




**Explanation:**

The output from each map function is a set of key value pairs. The outputs from each mapper are reorganized by keys when all mappers are complete and the job transitions from the map phase to the reduce phase. The reduce function is applied and we can expect one reducer per unique key. 


*Simple example of map outputs:*

- mapper1 output: {'A' : 10, 'B' : 7 }
- mapper2 output: {'A' : 5, 'C' : 5}
- mapper3 output: {'B' : 2, 'C' : 9}

The data are reorganized as follows, with 3 reducers (pseudocode):

- reduce(outKey='A', interValues=[10,5])
- reduce(outKey='B', interValues=[7 , 2])
- reduce(outKey='C', interValues=[5 , 9])

What we choose to do with outKey and interValues is up to the user. To train models in parallel, one per manufacturer (outKey), the metric values (interValues) in each reducer are:

1. concatenated to form the trainingDataSet
2. used to train the model (training pipeline provided from job.context)
3. the trained model is assigned a recognizable identifier (above, we use the model.id_outKey before upserting the model to the platform.

### D. MapReduce job specifications <a class="anchor" id="1D">

Now that we have defined our MLPipeline, map, and reduce functions - the MapReduce job specification `JSMapReduceSpec` pulls it all together.
    
> **Important Note**: Ensure that you have the type `MLTrainingJobSpecTemp` provisioned to your environment by running `c3ShowType(MLTrainingJobSpecTemp)` in the static console. This is a type in the starter code package that should have been provided to you and provisioned to your tenant/tag environment.
    

 ![image](http://learnc3.ai/wp-content/uploads/2020/01/Spec.png)

 
**Explanation:**

    
1\. `targetType` - Map reduce jobs always anchor on one type. What data are we reading and splitting up for processing? In this case, we want the job to use data from SmartBulb. 

2\. `include` - SmartBulb has many fields. Here, we are specifying precisely the fields required to accomplish our task. By default, the map/reduce functions will not be able to access fields other than `id`.

3\. `map` and `reduce` - after defining functions in console, these reference the functions by name

4\. `batchSize` - this is the number of SmarbBulb instances to be passed into each mapper. By default, batchSize is 10,000. Because we're using a small data set, we've set this to 1 so that 100 instances of SmartBulb will result in 100 mappers being spawned to demonstrate the concept. 

5\. `context` - Through context, we can pass any information we want to into the map reduce job. In the above, we pass an object composed of evalMetric parameters (start, end, interval), the list of features (metrc names) and target label (also a metric), and the MLSerialPipeline we defined previously. 

6\. `doNotAbortOnError` - defines whether or not the map reduce job should be cancelled if any errors are encountered.

The last line of code, `job = JS.mapReduce(spec)`, launches the MapReduce job. The function call returns immediately with the job reference which we store into the variable `job`. As you soon see, we'll use `job` to monitor and troubleshoot the map reduce execution.

> Tip: use c3ShowType(JSMapReduceSpec) for full explanation of all available options

# C3DS Monitor, Debug Jobs \& Queues <a class="anchor" id="part2">

## Section 2 - Launching the MapReduce Job \& Debugging <a class="anchor" id="2"></a>

The script to launch this job is in the folder with the assignment under the name `src/jobs/JSMapReduceTraining_BUGGY.js`. As you can see by the name, we have planted some errors in this script - specifically in the `mapFunction` and `reduceFunction`. 
    
The goal of this assignment is to help you get familiar with debugging queues and distributed jobs.

Run the script in a snippet or in console and perform `c3Grid(JSMapReduceJob.fetch())`. You will see a screen like:

![image](http://learnc3.ai/wp-content/uploads/2020/01/MRJob.png)

>**Helpful Debugging Tips**: Note that we added a filter here (`filter: "contains(id,'SMBLB15')",`) for debugging purpose so that the job only runs on one `SmartBulb` and is faster to do iterations to see what's not working.

#### Here are some additional helpful hints to debug jobs in console:

- After you have executed the code in `src/jobs/JSMapReduceTraining_BUGGY.js` in console, the last line of code in that script will have assigned the current execution of the JSMapReduceJob to the variable "job". As such, this variable can be used to inspect the state of the job which is running in the background on the cluster. Try these commands:

```
// return the unique id representing the JSMapReduce job
job.id 

// Get information about the running state of the job. 
job.status()

// From the output:
// "step" indicates whether job is in inital, readInput, map, or reduce phase.
// "status" indicates the overall job status, such as running, failing, or completed. See c3ShowType(JobStatusEnum) to see possible outputs representing the lifecycle of map reduce jobs

```
- Monitor `c3Grid(InvalidationQueue.countAll())` and you should see, after the intial run, values in the `Failed` column
- In console, look at the errors using `c3QErrs(MapReduceQueue)` and then look carefully at the code. You will see a very large and intimidating stack trace -- focus on the *top* of the stack -- so scroll up and read through the first 5-15 lines to find key messages that might give you a hint. 

- There is 1 line to fix in the mapFunction and 3 lines to fix in the reduceFunction. 

**In the map function:**

Look carefully at the `spec` for the `evalMetrics` call and compare it to your previous work evaluating metrics in the Metrics assignment. 

- Fix the error. Before running the job again, use `c3QErrorsClear(MapReduceQueue)` to clear the previous errors to avoid possible confusion. 

If you have resolved the error in the mapFunction, you'll hit the next error in the reduceFunction. 

**In the reduce function**:

The error will look like this:

```
TypeError: Cannot call method "train" of undefined (JSMapReduceJob_reduce.js#3(eval)#5)
```

There is a lot of useful debugging information here. Specifically, the error is in the reduce phase of JSMapReduceJob on line number #5

There is a minor typo on line 5. You'll need to fix this in a few other places too (line numbers help!). Continue to use `c3QErrorsClear(MapReduceQueue)` between runs until you've cleared all the errors.

- If you have any other problems, check the data loaded in your environment to ensure you have 100 SmartBulbs, and that all `SmartBulbMeasurementSeries`, and `SmartBulbMeasurements` correctly loaded. 

## Section 3 -  Run the Full MapReduce Job <a class="anchor" id="3"></a>

Now that you have a version with no errors of the MapReduce job, in the specification of the job, remove the line `filter: "contains(id,'SMBLB15')",` so that it does not only run on one `SmartBulb` but on all of them. If you then run the code again and wait for the MapReduce job to complete, you should see four different models when you run:

`c3Grid(MLSerialPipeline.fetch({filter: "contains(id, 'logisticRegressionMR')"}))`

![image](http://learnc3.ai/wp-content/uploads/2020/01/4models.png)


## Section 4 - Try Other Jobs (Optional) <a class="anchor" id="4"></a>

##### Calc Fields Jobs

- Use `c3ShowType(SmartBulb)` to find a calculated field

- Use `c3ShowType(RefreshCalcFieldsSpec)` to understand how to use SmartBulb.refreshCalcFields()

- Run `SmartBulb.refreshCalcFields()` and monitor the jobs with `c3Grid(InvalidationQueue.countAll())` 


##### Normalization Jobs

- Use `c3ShowType(RefreshNormalizationSpec)` to understand how to use `SmartBulbMeasurementSeries.refreshNormalization()`

- Run `SmartBulbMeasurementSeries.refreshNormalization()` and monitor the jobs with `c3Grid(InvalidationQueue.countAll())`

## Conclusion

Congratulations! You have successfully monitored and debugged jobs and queues On the C3 AI Suite. First, you developed three machine learning models on the fly, used jobs to manage these processes, monitored the associated jobs and queues, and finished with debugging.

## Make sure to CLOSE AND HALT  (from bottom of the File menu at the top) this notebook when you leave.

## Appendix - Kernel Installation Instructions <a class="anchor" id="apdx">
    
If the **`py-pythonEnv`** kernel is not available, perform the following:
1. Select the 'Kernel' tab at the top of this notebook, then select 'Manage Kernels';
2. Scroll to find `py-pythonEnv`, and click the 'Install' button;
> Note: This installation will "fail", but don't worry, exit the Manage Kernels UI and return to the main notebook.
3. Navigate to the 'File' tab located at the top of the notebook, and select 'Close and Halt'. This will close the notebook. Re-open it.
4. You should notice, in the upper-right corner, the kernel is now set correctly to `py-pythonEnv`. If it is not, navigate to the Kernel tab, then select 'Change Kernel' and select the appropriate option.